In [18]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings,HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.retrievers import MergerRetriever
from langchain.document_transformers import LongContextReorder
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
import os
import chromadb
load_dotenv()

True

## Get the Embedding Model

In [3]:
#get the embedding model
model_name ="BAAI/bge-large-en"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceBgeEmbeddings(model_name=model_name,
                                      model_kwargs=model_kwargs,
                                      encode_kwargs=encode_kwargs)
print("Embedding model loaded") 

c:\Maran\StudyMaterials\Git\LangChain\Retriever-MergerRetriever-LOTR\venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


Embedding model loaded


In [25]:
loader_constution_file = PyPDFLoader("data/constitution.pdf")
documents_set1 = loader_constution_file.load()
text_splitter_1 = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts_1 = text_splitter_1.split_documents(documents_set1)

# Create and store vector Store

In [26]:
file1_vec_store = Chroma.from_documents(texts_1,embedding=embeddings,
                                        collection_metadata={"hnsw:space":"cosine"},
                                        persist_directory="store/store_file")


### Load Vector Store

In [27]:
load_vector_store = Chroma(persist_directory="store/store_file", embedding_function=embeddings)

### Init Merge Retriever and perform Semantic Search

In [32]:
retriever_constituion_file = load_vector_store.as_retriever(search_type="similarity",search_kwargs={"k":3})


In [33]:
lotr = MergerRetriever(retrievers=[retriever_constituion_file])
lotr

MergerRetriever(retrievers=[VectorStoreRetriever(tags=['Chroma', 'HuggingFaceBgeEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000001872785D790>, search_kwargs={'k': 3})])

In [34]:
query = "what's the power of Congress?"

In [35]:
docs =lotr.get_relevant_documents(query)
docs

[Document(metadata={'page': 3, 'source': 'data/constitution.pdf'}, page_content='presented to the President of the United States; and before \nthe Same shall take Effect, shall be approved by him, or be- \ning disapproved by him, shall be repassed by two thirds of \nthe Senate and House of Representatives, according to the \nRules and Limitations prescribed in the Case of a Bill. \nSECTION. 8 \nThe Congress shall have Power To lay and collect Taxes, \nDuties, Imposts and Excises, to pay the Debts and provide \nfor the common Defence and general Welfare of the United \nStates; but all Duties, Imposts and Excises shall be uniform \nthroughout the United States;  \nTo borrow Money on the credit of the United States; \nTo regulate Commerce with foreign Nations, and among \nthe several States, and with the Indian  Tribes; \nTo establish an uniform Rule of Naturalization, and uni- \nform Laws on the subject of Bankruptcies throughout the \nUnited States; \nTo coin Money, regulate the Value t